In [7]:
# Change here using YOUR own first and last names
fn1 = "Jiale"
ln1 = "KANG"
filename = "_".join(map(lambda s: s.strip().lower(),
["SD-TSIA204_lab2", ln1, fn1])) + ".ipynb"

# 1)

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error as sklearn_mse
from sklearn.metrics import r2_score as sklearn_r2
import numpy as np

data_set = fetch_california_housing()
X = data_set.data
y = data_set.target

X = np.c_[np.ones(X.shape[0]), X]

# theta = (X^T X)^(-1) X^T y
theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

def compute_MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def compute_R2(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

y_pred = X.dot(theta)

my_mse = compute_MSE(y, y_pred)
my_r2 = compute_R2(y, y_pred)

sklearn_mse = sklearn_mse(y, y_pred)
sklearn_r2 = sklearn_r2(y, y_pred)

print("my MSE:", my_mse, "Sklearn MSE:", sklearn_mse)
print("my R2 :", my_r2, "Sklearn R2 :", sklearn_r2)

my MSE: 0.524320986184607 Sklearn MSE: 0.524320986184607
my R2 : 0.6062326851998052 Sklearn R2 : 0.6062326851998052


In [9]:
from scipy import stats

# Compute the residuals and their variance
residuals = y - y_pred
sigma_squared = np.mean(residuals ** 2)

# Compute the standard errors
XtX_inv = np.linalg.inv(X.T.dot(X))
standard_errors = np.sqrt(np.diag(sigma_squared * XtX_inv))

z = stats.norm.ppf(0.995)
confidence_intervals = [(theta[i] - z * standard_errors[i], theta[i] + z * standard_errors[i]) for i in range(len(theta))]

for i, ci in enumerate(confidence_intervals):
    print(f"Coefficient {i}: 99% CI = {ci}")


Coefficient 0: 99% CI = (-38.63874945118044, -35.24509096143688)
Coefficient 1: 99% CI = (0.42588540249626045, 0.4475011837773342)
Coefficient 2: 99% CI = (0.008286488997305977, 0.010585067069474924)
Coefficient 3: 99% CI = (-0.12247846827219933, -0.09216561451536173)
Coefficient 4: 99% CI = (0.5726106847258521, 0.7175207023358563)
Coefficient 5: 99% CI = (-1.6206846472410517e-05, 8.25406763075563e-06)
Coefficient 6: 99% CI = (-0.0050417663608660936, -0.0025313189491208005)
Coefficient 7: 99% CI = (-0.43984823431362086, -0.4027805207223839)
Coefficient 8: 99% CI = (-0.4539129628944105, -0.4151145464347442)


# 2)

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("data.csv", sep=",", header=None)

X_train, X_test, y_train, y_test = train_test_split(data.drop(210, axis=1), data[210], test_size=0.25, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def fit_OLS(X,y):
    theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    return theta

# fit OLS using LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_theta = np.r_[lr.intercept_, lr.coef_]


In [45]:
from scipy.stats import t

def choose_single_variable(X,y):
    results = pd.DataFrame(columns=['feature', 'p_value'])
    
    # Loop over all columns in X
    for column in range(X.shape[1]):
        X_column = X[:,column].reshape(-1,1)
        # Fit model
        model = LinearRegression().fit(X_column, y)
        # Calculate t-statistic
        t_stat = model.coef_ / (np.sqrt(compute_MSE(y, model.predict(X_column)) / ((X_column.var()) * (len(y) - 2))))
        # Calculate p-value
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=len(y)-2))
        # Append results use pandas.concat
        results = pd.concat([results, pd.DataFrame({'feature': [column], 'p_value': p_value})])
    
    # Return the feature with the smallest p-value
    return results.sort_values('p_value').iloc[0]

choose_single_variable(X_train_scaled, y_train)

feature    167
p_value    0.0
Name: 0, dtype: object

In [47]:
def forward_selection(X, y, significance_level=0.05):

    included_features = []
    while True:
        remaining_features = X.columns.difference(included_features)
        new_feature = choose_single_variable(X[remaining_features], y)
        if new_feature['p_value'] < significance_level:
            included_features.append(new_feature['feature'])
            print(f"Feature {new_feature['feature']} with p-value {new_feature['p_value']} was included.")
        else:
            break
    return included_features

forward_selection(X_train_scaled, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'